In [13]:
import h5pyd
import numpy as np


In [14]:
# executed by workers - add random values to cells
# n is the id for this worker
def add_points(n):
    # use 'a' since we'll be reading and writing to the domain
    f = h5pyd.File("pi_calc.test.hdfgroup.org", "a", endpoint="http://127.0.0.1:5000")
    g = f['pi']
    # get the number of workers
    num_workers = g.attrs['workers']
    pts = g['pts']
    # eacher worker will write count points
    count = pts.shape[0] / num_workers
    #arr = np.random.rand(n,2)
    for i in range(count):
        # update one row
        # In a real program, we'd want to update a batch of elements at a time
        pts[i*num_workers + n] = (rd.random(), rd.random())
    return count
        
 


In [15]:
# create a new domain (file)
f = h5pyd.File("pi_calc.test.hdfgroup.org", "w", endpoint="http://127.0.0.1:5000")
g = f.create_group('pi')
g.attrs['workers'] = 4
NUM = 1200
# create the dataset
dset = g.create_dataset('pts', (NUM,2), dtype='f8')



In [16]:
# setup the workers
from IPython.parallel import Client

c = Client()
dview = c.direct_view()
dview.execute('import random as rd')
dview.execute('import h5pyd')
dview.execute('import numpy as np')
c.block = True
c.ids


[0, 1, 2, 3]

In [17]:
# have workers run add_points
c[:].map(add_points, range(4))

[300, 300, 300, 300]

In [18]:
# tabulate results
count = 0
pts = dset[...]
for i in range(NUM):
    x = pts[i,0]
    y = pts[i,1]
    if x*x + y*y <= 1:
        count += 1
        
(count * 4.0)/NUM
    

3.15